In [ ]:
from google.colab import files

In [ ]:
!pip -q install kaggle pyspark pyarrow

In [ ]:
files.upload()

Saving tokens.ts to tokens.ts


{'tokens.ts': b'tokens\nKGAT_1943f2f81f04e39172723dd31c2ac3c0\n\nexport KAGGLE_API_TOKEN=KGAT_1943f2f81f04e39172723dd31c2ac3c0\n\nkaggle competitions list'}

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


mv: cannot stat 'kaggle.json': No such file or directory


In [ ]:
!kaggle datasets list | head


ref                                                             title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
saidaminsaidaxmadov/chocolate-sales                             Chocolate Sales                                         468320  2026-01-04 14:23:35.490000          12469        211  1.0              
sonalshinde123/work-from-home-employee-burnout-dataset          Work From Home Employee Burnout Dataset                  27962  2026-01-31 03:28:22.227000            793         31  1.0              
rockyt07/social-media-user-analysis                             Social Media User Analysis                           247842357  2026-01-14 02:28:41.970000           7338        149  1.0              


In [ ]:
DATASET = "mohamedbakhet/amazon-books-reviews"

!mkdir -p /content/data
!kaggle datasets download -d {DATASET} -p /content/data --unzip

Dataset URL: https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews
License(s): CC0-1.0
100% 1.06G/1.06G [00:03<00:00, 250MB/s]
100% 1.06G/1.06G [00:03<00:00, 302MB/s]


In [ ]:
!ls -lah /content/data


total 2.9G
drwxr-xr-x 2 root root 4.0K Feb  3 20:35 .
drwxr-xr-x 1 root root 4.0K Feb  3 20:34 ..
-rw-r--r-- 1 root root 173M Feb  3 20:35 books_data.csv
-rw-r--r-- 1 root root 2.7G Feb  3 20:35 Books_rating.csv


In [ ]:
DATA_DIR = "/content/data"
RATINGS_CSV = f"{DATA_DIR}/Books_rating.csv"
BOOKS_CSV = f"{DATA_DIR}/books_data.csv"

# just to save runtime
USE_FULL_DATA = False
SAMPLE_FRAC = 0.20
SEED = 12345


In [ ]:
K_EVAL = 10 # report size
MAX_USERS_EVAL = 2000 # cap for speed
MIN_TEST_POS = 1 # user must have at least this many test items
MIN_TRAIN_ITEMS = 3 # need some history to recommend


In [ ]:
# Start the SPark session

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("amd-recsys-books")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("WARN")


In [ ]:
# Load book ratings into Spark:
from pyspark.sql import functions as F

ratings = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(RATINGS_CSV)
)

ratings.printSchema()
ratings.show(5, truncate=False)


root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: string (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)

+----------+------------------------------+-----+--------------+--------------------------------------+------------------+------------+-----------+-----------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# check the columns to understand what's inside

cols = {c.lower(): c for c in ratings.columns}
cols

{'id': 'Id',
 'title': 'Title',
 'price': 'Price',
 'user_id': 'User_id',
 'profilename': 'profileName',
 'review/helpfulness': 'review/helpfulness',
 'review/score': 'review/score',
 'review/time': 'review/time',
 'review/summary': 'review/summary',
 'review/text': 'review/text'}

In [ ]:
# Normalize and select core columns
from pyspark.sql import functions as functions

score_string = functions.regexp_extract(functions.col("review/score"), r"([0-9]+(\.[0-9]+)?)", 1)

ratings_0 = (
    ratings
    .select(
        functions.col("User_id").alias("user_id"),
        functions.col("Id").alias("book_id"),
        # safe numeric extraction for odd strings
        functions.when(score_string != "", score_string.cast("double")).otherwise(F.lit(None).cast("double")).alias("rating"),
        functions.col("Title").alias("title"),
    )
    .dropna(subset=["user_id", "book_id", "rating"])
)

ratings_0.select("rating").describe().show()
ratings_0.show(3, truncate=False)


+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|           2433202|
|   mean|2029.6013891366192|
| stddev|1580342.9802411501|
|    min|               0.0|
|    max|        1.295568E9|
+-------+------------------+

+--------------+----------+------+------------------------------+
|user_id       |book_id   |rating|title                         |
+--------------+----------+------+------------------------------+
|AVCGYZL8FQQTD |1882931173|4.0   |Its Only Art If Its Well Hung!|
|A30TK6U7DNS82R|0826414346|5.0   |Dr. Seuss: American Icon      |
|A3UH4UZ4RSVO82|0826414346|5.0   |Dr. Seuss: American Icon      |
+--------------+----------+------+------------------------------+
only showing top 3 rows


In [ ]:
# clear bad formed data for ratings, first got 10^8 score

# keep only realistic star ratings
ratings_1 = ratings_0.filter((F.col("rating") >= 1.0) & (F.col("rating") <= 5.0))

ratings_1.select("rating").describe().show()


+-------+-----------------+
|summary|           rating|
+-------+-----------------+
|  count|          2426292|
|   mean|4.222532572336718|
| stddev|1.183760589211074|
|    min|              1.0|
|    max|              5.0|
+-------+-----------------+



In [ ]:
ratings.select("review/score").show(5, truncate=False)
ratings.select("review/helpfulness").show(5, truncate=False)


+------------+
|review/score|
+------------+
|4.0         |
|5.0         |
|5.0         |
|4.0         |
|4.0         |
+------------+
only showing top 5 rows
+------------------+
|review/helpfulness|
+------------------+
|7/7               |
|10/10             |
|10/11             |
|7/7               |
|3/3               |
+------------------+
only showing top 5 rows


In [ ]:
# Sample and reduce sparcity, for speed

ratings = ratings_1
if not USE_FULL_DATA:
    ratings = ratings.sample(False, SAMPLE_FRAC, seed=SEED)

MIN_USER_RATINGS = 5
MIN_BOOK_RATINGS = 5

user_count = ratings.groupBy("user_id").count().withColumnRenamed("count", "user_count")
book_count = ratings.groupBy("book_id").count().withColumnRenamed("count", "book_count")

ratings = (
    ratings.join(user_count, "user_id")
     .join(book_count, "book_id")
     .filter((functions.col("user_count") >= MIN_USER_RATINGS) & (functions.col("book_count") >= MIN_BOOK_RATINGS))
     .select("user_id", "book_id", "rating", "title")
)

print("rows:", ratings.count())
print("users:", ratings.select("user_id").distinct().count())
print("books:", ratings.select("book_id").distinct().count())

ratings.cache()


rows: 85603
users: 9795
books: 13660


DataFrame[user_id: string, book_id: string, rating: double, title: string]

In [ ]:
from pyspark.sql import functions as F

ratings.groupBy("book_id").count().orderBy(F.col("count").desc()).show(20)
ratings.groupBy("user_id").count().orderBy(F.col("count").desc()).show(20)


+----------+-----+
|   book_id|count|
+----------+-----+
|0141804459|  178|
|B000GQG7D2|  171|
|B000NDSX6C|  168|
|B000C1X8JC|  168|
|0786135034|  167|
|B000F6H01Q|  165|
|B000GQG5MA|  164|
|B000GDLGSG|  164|
|B000ILIJE0|  164|
|B000Q032UY|  164|
|1844560333|  161|
|B000NWQXBA|  159|
|0451513967|  159|
|0435126075|  158|
|8188280046|  157|
|1901768945|  157|
|B000EVI8O0|  155|
|1566190932|  150|
|B000H9R1Q0|  148|
|B000PC54NG|  145|
+----------+-----+
only showing top 20 rows
+--------------+-----+
|       user_id|count|
+--------------+-----+
|A1D2C0WDCSHUWZ|  568|
|   AFVQZQ8PW0L|  290|
|A14OJS0VWMOSWO|  266|
|A20EEWWSFMZ1PN|  212|
|A1X8VZWTOG8IS6|  211|
| AHD101501WCN1|  184|
|A1K1JW1C5CUSUZ|  179|
|A1EKTLUL24HDG8|  145|
|A1G37DFO8MQW0M|  143|
|A1N1YEMTI9DJ86|  139|
|A3QVAKVRAH657N|  136|
| AHXAPVSHPJ6OJ|  132|
|A2F6N60Z96CAJI|  131|
|A1L43KWWR05PCS|  129|
|A1T17LMQABMBN5|  117|
|A319KYEIAZ3SON|  113|
|A3M174IC0VXOS2|  111|
| AJQ1S39GZBKUG|  109|
|A3LKWMM12AF0PU|  108|
|A28WJUJF6D2U

In [ ]:
# Baseline recommender by popularity
# Check it to compare later

MIN_REVIEWS_FOR_POPULARITY = 10

popular = (
    ratings.groupBy("book_id", "title")
     .agg(functions.count("*").alias("n"), functions.avg("rating").alias("average_rating"))
     .filter(functions.col("n") >= MIN_REVIEWS_FOR_POPULARITY)
     .orderBy(functions.col("average_rating").desc(), functions.col("n").desc())
)

popular.show(20, truncate=False)


+----------+-----------------------------------------------------------------------------------------+---+-----------------+
|book_id   |title                                                                                    |n  |average_rating   |
+----------+-----------------------------------------------------------------------------------------+---+-----------------+
|B000OZRZ90|Milton's Paradise Lost                                                                   |12 |5.0              |
|B000JWHH32|The Jungle Book (Companion Library)                                                      |11 |5.0              |
|1854596209|Hamlet (The Shakespeare Folios)                                                          |11 |5.0              |
|B0007J8XJ4|The varieties of religious experience: A study in human nature (Gifford lectures)        |10 |5.0              |
|B0007G64NO|The Screwtape letters & Screwtape proposes a toast (Time reading program special edition)|10 |5.0              |


In [ ]:
# Train test split
train_df, test_df = ratings.randomSplit([0.8, 0.2], seed=SEED)

train_df = train_df.cache()
test_df  = test_df.cache()

print("train:", train_df.count(), "test:", test_df.count())


train: 68307 test: 17296


Item-item colaborative filtering

In [ ]:
# build user to list of pairs item, rating from train data

# resilient distr dataset
train_rdd = train_df.select("user_id", "book_id", "rating").rdd

user_items = (
    train_rdd
    .map(lambda row: (row["user_id"], (row["book_id"], float(row["rating"]))))
    .groupByKey()
    .mapValues(list)
)


In [ ]:
# Generate item pairs per user and get stats aggregation

import itertools
import math
# safety cap for many reviews
MAX_ITEMS_PER_USER = 50

def make_pairs(items):
    # items are list of (book_id, rating)
    if len(items) > MAX_ITEMS_PER_USER:
        items = items[:MAX_ITEMS_PER_USER]  # deterministic cap, todo: improve
    out = []
    for (i, ri), (j, rj) in itertools.combinations(items, 2):
        if i == j:
            continue
        a, b = (i, j) if i < j else (j, i)
        ra, rb = (ri, rj) if i < j else (rj, ri)
        out.append(((a, b), (ra*rb, ra*ra, rb*rb, 1)))
    return out

pair_stats = (
    user_items
    .flatMap(lambda x: make_pairs(x[1]))
    .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1], x[2]+y[2], x[3]+y[3]))
)


In [ ]:
# Compute cosine similarity

MIN_COMMON_USERS = 2

similarities = (
    pair_stats
    .filter(lambda kv: kv[1][3] >= MIN_COMMON_USERS)
    .mapValues(lambda v: (
        v[0] / (math.sqrt(v[1]) * math.sqrt(v[2]) + 1e-12),  # cosine similarity
        v[3]  # common users
    ))
)

print("number of item pairs with common users >= min common users:", similarities.count())
print("sample similarity:", similarities.take(5))


number of item pairs with common users >= min common users: 22827
sample similarity: [(('1420926810', '1570020981'), (0.9374252720097401, 2)), (('1841490717', 'B000N7204Y'), (0.986025143961732, 3)), (('0792717848', 'B00005QTHG'), (0.8531145054348057, 3)), (('076240552X', 'B000MZCJUM'), (0.9999999999999929, 6)), (('B000LEPWGC', 'B000NUA82W'), (0.9938837346735968, 2))]


In [ ]:
BETA = 10  # larger = more conservative

def significance(sim, cnt, beta=BETA):
    shrink = cnt / (cnt + beta)
    sim_w = sim * shrink
    return sim_w, shrink, cnt


In [ ]:
# KNN per item, top

K = 30

neighbors = similarities.flatMap(lambda kv: [
    (kv[0][0], (kv[0][1],) + significance(kv[1][0], kv[1][1])),  # i -> (j, sim_w, shrink, cnt)
    (kv[0][1], (kv[0][0],) + significance(kv[1][0], kv[1][1]))   # j -> (i, sim_w, shrink, cnt)
])

def topk(xs, k=K):
    # t[1] is sim'
    xs = sorted(xs, key=lambda t: t[1], reverse=True)
    return xs[:k]

item_neighbors = neighbors.groupByKey().mapValues(lambda xs: topk(list(xs), K))

# collect is ok at this sample size
item_neighbors_map = dict(item_neighbors.collect())
print("items with neighbor lists:", len(item_neighbors_map))


items with neighbor lists: 4753


In [ ]:
# Broadcast!!

broadcast_neighbours = spark.sparkContext.broadcast(item_neighbors_map)

In [ ]:
train_ui = (
    train_df.groupBy("user_id", "book_id")
            .agg(functions.avg("rating").alias("rating"))
)

# Broadcast user history from train
user_history_map = dict(
    train_ui.rdd
    .map(lambda r: (r["user_id"], (r["book_id"], float(r["rating"]))))
    .groupByKey()
    .mapValues(list)
    .collect()
)

broadcast_history = spark.sparkContext.broadcast(user_history_map)
print("users in history:", len(user_history_map))


users in history: 9717


In [29]:
import math

def predict_one(user_id, target_item):
    history = broadcast_history.value.get(user_id, [])
    neighbours = broadcast_neighbours.value.get(target_item, [])
    if not history or not neighbours:
        return None

    history_map = dict(history)

    num = 0.0
    den = 0.0
    for j, sim_w, shrink, cnt in neighbours:
        if j in history_map:
            num += sim_w * history_map[j]
            den += abs(sim_w)

    if den < 1e-9:
        return None

    prediction = num / den
    return max(1.0, min(5.0, prediction))

predicted_test_true = (
    test_df.select("user_id","book_id","rating").rdd
    .map(lambda r: (predict_one(r["user_id"], r["book_id"]), float(r["rating"])))
    .filter(lambda x: x[0] is not None)
)

mse = predicted_test_true.map(lambda x: (x[0] - x[1])**2).mean()
rmse = math.sqrt(mse)

predicted_count = predicted_test_true.count()
test_count = test_df.count()
coverage = predicted_count / test_count

print("RMSE:", rmse)
print("Coverage:", coverage)
print("Predicted test points:", predicted_count, "/", test_count)


RMSE: 0.4100855556182296
Coverage: 0.38806660499537465
Predicted test points: 6712 / 17296


COverage improved significantly.
Now Baseline RMSE

In [30]:
import math
from pyspark.sql import functions as functions

global_mean = train_df.agg(functions.avg("rating")).first()[0]
baseline_mse = test_df.select("rating").rdd.map(lambda r: (float(r[0]) - global_mean)**2).mean()
baseline_rmse = math.sqrt(baseline_mse)

print("Global mean:", global_mean)
print("Baseline RMSE (global mean):", baseline_rmse)


Global mean: 4.2634137057695405
Baseline RMSE (global mean): 1.0191472853605235


In [31]:
# Top N recommendations for a user
book_titles = dict(
    ratings.select("book_id", "title")
           .dropna(subset=["title"])
           .dropDuplicates(["book_id"])
           .rdd.map(lambda r: (r["book_id"], r["title"]))
           .collect()
)

print("book titles len:", len(book_titles))


book titles len: 13659


In [35]:
def recommend_for_user(user_id, N=10):
    history = broadcast_history.value.get(user_id, [])
    # cold start
    if not history:
        return []

    seen = set(i for i, _ in history)
    # item -> (num, den, contribs)
    scores = {}

    for i, r_ui in history:
        for j, sim_w, shrink, count in broadcast_neighbours.value.get(i, []):
            if j in seen:
                continue
            num, den, c = scores.get(j, (0.0, 0.0, 0))
            num += sim_w * r_ui
            den += abs(sim_w)
            c += 1
            scores[j] = (num, den, c)

    predictions = []
    for j, (num, den, c) in scores.items():
        if den > 1e-9 and c >= 2:  # small filter makes output nicer
            prediction = max(1.0, min(5.0, num / den))
            predictions.append((j, prediction, c))

    predictions.sort(key=lambda x: (x[1], x[2]), reverse=True)
    return predictions[:N]


In [36]:
# Let's pick one of the users and ptint top rated

def top_rated_unique(user_id, n=10):
    history = broadcast_history.value.get(user_id, [])
    # sort by rating, then unique by book_id
    history = sorted(history, key=lambda x: x[1], reverse=True)
    seen = set()
    out = []
    for bid, r in history:
        if bid in seen:
            continue
        seen.add(bid)
        out.append((bid, r))
        if len(out) >= n:
            break
    return out

USER = "A1D2C0WDCSHUWZ"
for bid, r in top_rated_unique(USER, 10):
    print(f"{r:.1f}  {bid}  -  {book_titles.get(bid, '<no title>')}")


5.0  B000H9R1Q0  -  The Hobbit
5.0  B000Q032UY  -  The Hobbit or There and Back Again
5.0  9626346949  -  Little Women (Junior Classics)
5.0  B000H7EO2G  -  The Lord of the Rings Trilogy 3 Volumes
5.0  B0007NMYQ8  -  The Two Princesses of Bamarre
5.0  B000C1X8JC  -  Pride and Prejudice
5.0  B000F6H01Q  -  Pride and Prejudice
5.0  B000MPEZ9K  -  Emma
5.0  B000Q382GK  -  The Lion, the Witch and the Wardrobe: A Story for Children
5.0  0395051150  -  Emma (Riverside Editions)


In [37]:
# SO, he is hitting 5/5 often
# Let's check what user liked


recommendations = recommend_for_user(USER, 10)
for bid, prediction, c in recommendations:
    print(f"{prediction:.3f} (from {c} neighbour)  {bid}  -  {book_titles.get(bid, '<no title>')}")


5.000 (from 35 neighbour)  1566190932  -  Pride and Prejudice
5.000 (from 25 neighbour)  B000EVI8O0  -  Pride and Prejudice
5.000 (from 13 neighbour)  9626341823  -  A Christmas Carol (Classic Fiction)
5.000 (from 12 neighbour)  0140860282  -  Wuthering Heights (Penguin Audiobooks)
5.000 (from 10 neighbour)  B000Q08CDQ  -  Return of the King Being the Third Part of The Lord of the Rings
5.000 (from 9 neighbour)  B0006AQ4LI  -  Wuthering Heights
5.000 (from 8 neighbour)  1590862899  -  The Scarlet Letter
5.000 (from 8 neighbour)  B000N6DDJQ  -  The Scarlet Letter A Romance
5.000 (from 7 neighbour)  1593351348  -  Wuthering Heights
5.000 (from 7 neighbour)  0435126083  -  Wuthering Heights (New Windmill)


In [38]:
# Let's check abother user

USER = "A14OJS0VWMOSWO"
for bid, r in top_rated_unique(USER, 40):
    print(f"{r:.1f}  {bid}  -  {book_titles.get(bid, '<no title>')}")

recommendations = recommend_for_user(USER, 40)
for bid, prediction, c in recommendations:
    print(f"{prediction:.3f} (from {c} neighbour)  {bid}  -  {book_titles.get(bid, '<no title>')}")



5.0  B000PJLA96  -  Understanding Exposure
5.0  0873529871  -  MLA Handbook for Writers of Research Papers (Sixth Edition Large Print)
5.0  0394779959  -  Summer of My German Soldier
5.0  1574904558  -  Lauren's Story: An American Dog in Paris (Beeler Large Print Series)
5.0  1559270845  -  The Womanly Art of Breastfeeding
5.0  0192817728  -  The Secret Garden (Worlds Classics)
5.0  193011088X  -  Swing, Second Edition
5.0  B000NWKKAU  -  The A. B. C. Murders
5.0  0970540116  -  Flour Power: A Guide To Modern Home Grain Milling
5.0  1400053684  -  Countdown to Crisis: The Coming Nuclear Showdown with Iran
5.0  B000FFJSO4  -  Priestess Of Avalon
5.0  B000GLVRLY  -  Suzannes Diary For Nicholas
5.0  0921165730  -  Vitamin C for a Healthy Workplace
5.0  B0006Y0EN8  -  The running man
5.0  B00007FYA3  -  Practical Standards for Microsoft Visual Basic
5.0  B0006APKUO  -  Rebecca
5.0  0140298401  -  Forever Liesl: A Memoir of The Sound of Music
5.0  0971953880  -  When You Work for a Bully: A

In [40]:
# recommend for user and don’t clip at 5.0 for display
def recommend_for_user(user_id, N=10):
    history = broadcast_history.value.get(user_id, [])
    if not history:
        return []

    seen = set(i for i, _ in history)
    scores = {}

    for i, r_ui in history:
        for j, sim_w, shrink, count in broadcast_neighbours.value.get(i, []):
            if j in seen:
                continue
            num, den, c = scores.get(j, (0.0, 0.0, 0))
            num += sim_w * r_ui
            den += abs(sim_w)
            c += 1
            scores[j] = (num, den, c)

    predictions = []
    for j, (num, den, c) in scores.items():
        if den > 1e-9 and c >= 2:
            raw = num / den
            clipped = max(1.0, min(5.0, raw))
            predictions.append((j, raw, clipped, c))

    # sort by clipped then contribs
    predictions.sort(key=lambda x: (x[2], x[3]), reverse=True)
    return predictions[:N]

recommendations = recommend_for_user(USER, 200)
for bid, raw, clipped, c in recommendations:
    print(f"{clipped:.3f} (raw {raw:.3f}, from {c} neigh)  {bid}  -  {book_titles.get(bid, '<no title>')}")


5.000 (raw 5.000, from 5 neigh)  140505347X  -  Alice's Adventures in Wonderland
5.000 (raw 5.000, from 4 neigh)  B000H9R1Q0  -  The Hobbit
5.000 (raw 5.000, from 4 neigh)  B000GQG5MA  -  The Hobbit; Or, There and Back Again
5.000 (raw 5.000, from 4 neigh)  B000CEDH40  -  Lewis Carroll's Alice in Wonderland
5.000 (raw 5.000, from 4 neigh)  0904724719  -  Alice in Wonderland (Tell tales)
5.000 (raw 5.000, from 4 neigh)  0786261080  -  Alice's Adventures in Wonderland
5.000 (raw 5.000, from 4 neigh)  B000IOKF8U  -  "Alice""s Adventures in Wonderland"
5.000 (raw 5.000, from 4 neigh)  068983375X  -  Alice's Adventures in Wonderland (Aladdin Classics)
5.000 (raw 5.000, from 3 neigh)  0451519582  -  Wuthering Heights (Signet classics)
5.000 (raw 5.000, from 3 neigh)  9626346949  -  Little Women (Junior Classics)
5.000 (raw 5.000, from 3 neigh)  B000NNOTXI  -  To Kill a Mocking Bird
5.000 (raw 5.000, from 3 neigh)  B000GY0PV4  -  The Two Towers
5.000 (raw 5.000, from 3 neigh)  B000TZ19TC  -  

In [41]:
# Mean centered item item
from pyspark.sql import functions as F

user_mean_map = dict(
    train_df.groupBy("user_id")
            .agg(F.avg("rating").alias("mu"))
            .rdd.map(lambda r: (r["user_id"], float(r["mu"])))
            .collect()
)
bum = spark.sparkContext.broadcast(user_mean_map)

print("users with mean:", len(user_mean_map))


users with mean: 9717


In [44]:
# Mean centered predictor

def predict_one_centered(user_id, target_item):
    history = broadcast_history.value.get(user_id, [])
    neighbours = broadcast_neighbours.value.get(target_item, [])
    if not history or not neighbours:
        return None

    mu = bum.value.get(user_id, 4.0)
    history_map = dict(history)

    num = 0.0
    den = 0.0
    contribution = 0

    for j, sim_w, shrink, count in neighbours:
        if j in history_map:
            num += sim_w * (history_map[j] - mu)   # deviation from user mean
            den += abs(sim_w)
            contribution += 1

    if den < 1e-9:
        return None

    prediction = mu + num / den
    prediction = max(1.0, min(5.0, prediction))
    return prediction, contribution


In [45]:
# recompute RMSE and coverage with mean centered predictor
import math

predicted_true = (
    test_df.select("user_id","book_id","rating").rdd
    .map(lambda r: (predict_one_centered(r["user_id"], r["book_id"]), float(r["rating"])))
    .filter(lambda x: x[0] is not None)
    .map(lambda x: (x[0][0], x[1]))  # keep only pred, true
)

mse = predicted_true.map(lambda x: (x[0] - x[1])**2).mean()
rmse_centered = math.sqrt(mse)

predicted_count = predicted_true.count()
test_count = test_df.count()
coverage_centered = predicted_count / test_count

print("Centered RMSE:", rmse_centered)
print("Centered Coverage:", coverage_centered)
print("Predicted:", predicted_count, "/", test_count)


Centered RMSE: 0.4100855556182296
Centered Coverage: 0.38806660499537465
Predicted: 6712 / 17296


In [46]:
# Mean centered top N recommendations
def recommend_for_user_centered(user_id, N=10, min_contrib=2):
    history = broadcast_history.value.get(user_id, [])
    if not history:
        return []

    mu = bum.value.get(user_id, 4.0)
    seen = set(i for i, _ in history)
    history_map = dict(history)

    scores = {}  # item -> (num, den, contrib)
    for i, r_ui in history:
        for j, sim_w, shrink, count in broadcast_neighbours.value.get(i, []):
            if j in seen:
                continue
            num, den, c = scores.get(j, (0.0, 0.0, 0))
            num += sim_w * (history_map[i] - mu)   # use deviation
            den += abs(sim_w)
            c += 1
            scores[j] = (num, den, c)

    predictions = []
    for j, (num, den, c) in scores.items():
        if den > 1e-9 and c >= min_contrib:
            prediction = mu + num / den
            prediction = max(1.0, min(5.0, prediction))
            predictions.append((j, prediction, c))

    predictions.sort(key=lambda x: (x[1], x[2]), reverse=True)
    return predictions[:N]


In [47]:
USER = "A3M174IC0VXOS2"
recommendations = recommend_for_user_centered(USER, 100)

for bid, prediction, c in recommendations:
    print(f"{prediction:.3f} (from {c} neigh)  {bid}  -  {book_titles.get(bid, '<no title>')}")


5.000 (from 4 neigh)  0451519582  -  Wuthering Heights (Signet classics)
5.000 (from 4 neigh)  0435126075  -  Pride & Prejudice (New Windmill)
5.000 (from 4 neigh)  0140860282  -  Wuthering Heights (Penguin Audiobooks)
5.000 (from 4 neigh)  0134354575  -  Wuthering Heights
5.000 (from 3 neigh)  B000FFQ85G  -  Jane Eyre / Wuthering Heights
5.000 (from 3 neigh)  B000F6H01Q  -  Pride and Prejudice
5.000 (from 3 neigh)  B000P4Q3JS  -  Wuthering Heights (The Franklin Library)
5.000 (from 3 neigh)  B0007C10MS  -  The great Gatsby (Leading English literature library)
5.000 (from 3 neigh)  0140351310  -  Jane Eyre: Complete and Unabridged (Puffin Classics)
5.000 (from 3 neigh)  0451518845  -  Jane Eyre (Signet classics)
5.000 (from 3 neigh)  0681994851  -  Jane Eyre
5.000 (from 3 neigh)  0435126083  -  Wuthering Heights (New Windmill)
5.000 (from 3 neigh)  1593355548  -  Wuthering Heights
5.000 (from 3 neigh)  1591090245  -  Wuthering Heights
5.000 (from 3 neigh)  1901768945  -  Pride and Prej

In [48]:
def explain_recommendation(user_id, target_item, top_m=3):
    history = broadcast_history.value.get(user_id, [])
    mu = bum.value.get(user_id, 4.0)
    history_map = dict(history)

    reasons = []
    for j, sim_w, shrink, cnt in broadcast_neighbours.value.get(target_item, []):
        if j in history_map:
            contrib = sim_w * (history_map[j] - mu)
            reasons.append((j, sim_w, shrink, cnt, history_map[j], contrib))

    reasons.sort(key=lambda x: abs(x[5]), reverse=True)  # contrib is now x[5]
    return reasons[:top_m]


USER = "A3M174IC0VXOS2"
recommendations = recommend_for_user_centered(USER, 10)

for bid, prediction, c in recommendations:
    print(f"\n{prediction:.3f} (from {c} neigh)  {book_titles.get(bid, bid)}")
    for j, sim_w, shrink, cnt, rj, contrib in explain_recommendation(USER, bid, top_m=3):
        print(
            f"  because: {book_titles.get(j, j)} | "
            f"sim'={sim_w:.3f} (shrink={shrink:.3f}, common={cnt}), your_rating={rj:.1f}, contrib={contrib:+.3f}"
        )



5.000 (from 4 neigh)  Wuthering Heights (Signet classics)
  because: Wuthering Heights | sim'=0.703 (shrink=0.714, common=25), your_rating=5.0, contrib=+0.178
  because: Wuthering Heights (Riverside editions) | sim'=0.663 (shrink=0.667, common=20), your_rating=5.0, contrib=+0.167

5.000 (from 4 neigh)  Pride & Prejudice (New Windmill)
  because: Jane Eyre (Simple English) | sim'=0.286 (shrink=0.286, common=4), your_rating=5.0, contrib=+0.072
  because: Jane Eyre (Large Print) | sim'=0.285 (shrink=0.286, common=4), your_rating=5.0, contrib=+0.072

5.000 (from 4 neigh)  Wuthering Heights (Penguin Audiobooks)
  because: Wuthering Heights | sim'=0.724 (shrink=0.730, common=27), your_rating=5.0, contrib=+0.183
  because: Wuthering Heights (Riverside editions) | sim'=0.683 (shrink=0.688, common=22), your_rating=5.0, contrib=+0.173
  because: Jane Eyre (Simple English) | sim'=0.374 (shrink=0.375, common=6), your_rating=5.0, contrib=+0.094

5.000 (from 4 neigh)  Wuthering Heights
  because: W

In [49]:
from pyspark.sql import functions as F

train_df.groupBy("rating").count().orderBy("rating").show()


+------+-----+
|rating|count|
+------+-----+
|   1.0| 2248|
|   2.0| 3007|
|   3.0| 7477|
|   4.0|17347|
|   5.0|38228|
+------+-----+



In [50]:
USER = "A3M174IC0VXOS2"
print("user mean:", bum.value.get(USER))


user mean: 4.747252747252747


In [51]:
def recommend_for_user_ranked(user_id, N=10, min_contrib=2, support_bonus=0.02):
    hist = broadcast_history.value.get(user_id, [])
    if not hist:
        return []

    mu = bum.value.get(user_id, 4.0)
    seen = set(i for i, _ in hist)
    hist_map = dict(hist)

    scores = {}  # item -> (num, den, contrib)
    for i, r_ui in hist:
        for j, sim_w,  shrink, count in broadcast_neighbours.value.get(i, []):
            if j in seen:
                continue
            num, den, c = scores.get(j, (0.0, 0.0, 0))
            num += sim_w * (hist_map[i] - mu)
            den += abs(sim_w)
            c += 1
            scores[j] = (num, den, c)

    out = []
    for j, (num, den, c) in scores.items():
        if den > 1e-9 and c >= min_contrib:
            pred = mu + num / den
            pred = max(1.0, min(5.0, pred))
            rank_score = pred + support_bonus * c   # tie-break
            out.append((j, pred, c, rank_score))

    out.sort(key=lambda x: x[3], reverse=True)
    return out[:N]


In [52]:
USER = "A3M174IC0VXOS2"
recommendations = recommend_for_user_ranked(USER, N=15)

for bid, prediction, c, score in recommendations:
    print(f"score={score:.3f} pred={prediction:.3f} (c={c})  {book_titles.get(bid, bid)}")


score=5.080 pred=5.000 (c=4)  Wuthering Heights (Signet classics)
score=5.080 pred=5.000 (c=4)  Pride & Prejudice (New Windmill)
score=5.080 pred=5.000 (c=4)  Wuthering Heights (Penguin Audiobooks)
score=5.080 pred=5.000 (c=4)  Wuthering Heights
score=5.060 pred=5.000 (c=3)  Jane Eyre / Wuthering Heights
score=5.060 pred=5.000 (c=3)  Pride and Prejudice
score=5.060 pred=5.000 (c=3)  Wuthering Heights (The Franklin Library)
score=5.060 pred=5.000 (c=3)  The great Gatsby (Leading English literature library)
score=5.060 pred=5.000 (c=3)  Jane Eyre: Complete and Unabridged (Puffin Classics)
score=5.060 pred=5.000 (c=3)  Jane Eyre (Signet classics)
score=5.060 pred=5.000 (c=3)  Jane Eyre
score=5.060 pred=5.000 (c=3)  Wuthering Heights (New Windmill)
score=5.060 pred=5.000 (c=3)  Wuthering Heights
score=5.060 pred=5.000 (c=3)  Wuthering Heights
score=5.060 pred=5.000 (c=3)  Pride and Prejudice


TopN evaluation

In [53]:
# Build per user train items

# train items per user
train_user_items_df = (
    train_df.select("user_id", "book_id")
            .dropDuplicates()
            .groupBy("user_id")
            .agg(functions.collect_set("book_id").alias("train_items"))
)

# test positives per user
test_user_items_df = (
    test_df.select("user_id", "book_id")
           .dropDuplicates()
           .groupBy("user_id")
           .agg(functions.collect_set("book_id").alias("test_items"))
)

# join and filter users
evaluated_users_df = (
    train_user_items_df.join(test_user_items_df, on="user_id", how="inner")
    .withColumn("n_train", functions.size("train_items"))
    .withColumn("n_test", functions.size("test_items"))
    .filter((functions.col("n_train") >= MIN_TRAIN_ITEMS) & (functions.col("n_test") >= MIN_TEST_POS))
)

print("Eligible evaluated users:", evaluated_users_df.count())

# sample users for speed
evaluated_users_sample = evaluated_users_df.limit(MAX_USERS_EVAL).collect()
print("Evaluating on users:", len(evaluated_users_sample))


Eligible evaluated users: 6351
Evaluating on users: 2000


In [56]:
# Recommendation function that excludes seen items
def recommend_topk_from_train_items(train_items, k=K_EVAL):
    # train_items: list of item_ids the user has in train
    seen = set(train_items)
    scores = {}  # item -> score

    # Spread scores from each seen item to its neighbors
    for i in seen:
        for j, sim_w, shrink, count in broadcast_neighbours.value.get(i, []):
            if j in seen:
                continue
            scores[j] = scores.get(j, 0.0) + sim_w

    # Top-k by score
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [item for item, s in ranked[:k]]


In [57]:
# Compute metrics: precision, recall etc

import math

# Normalized Discounted Cumulative Gain
def ndcg_at_k(recommended, positives, k):
    # binary relevance: 1 if in positives
    dcg = 0.0
    for idx, item in enumerate(recommended[:k], start=1):
        if item in positives:
            dcg += 1.0 / math.log2(idx + 1)

    # ideal DCG: all positives ranked first
    ideal_hits = min(len(positives), k)
    idcg = sum(1.0 / math.log2(i + 1) for i in range(1, ideal_hits + 1))
    return dcg / idcg if idcg > 0 else 0.0

hits = 0
sum_prec = 0.0
sum_rec = 0.0
sum_ndcg = 0.0
n_users = 0

for row in evaluated_users_sample:
    user = row["user_id"]
    train_items = row["train_items"]
    test_items = set(row["test_items"])

    recommendations = recommend_topk_from_train_items(train_items, k=K_EVAL)
    if not recommendations:
        continue

    recommendations_set = set(recommendations)
    n_hit = len(recommendations_set.intersection(test_items))

    # metrics
    hit_rate = 1 if n_hit > 0 else 0
    precision = n_hit / K_EVAL
    recall = n_hit / len(test_items)

    hits += hit_rate
    sum_prec += precision
    sum_rec += recall
    sum_ndcg += ndcg_at_k(recommendations, test_items, K_EVAL)
    n_users += 1

print("Users evaluated:", n_users)
print(f"HitRate@{K_EVAL}: {hits / n_users:.4f}")
print(f"Precision@{K_EVAL}: {sum_prec / n_users:.4f}")
print(f"Recall@{K_EVAL}: {sum_rec / n_users:.4f}")
print(f"NDCG@{K_EVAL}: {sum_ndcg / n_users:.4f}")


Users evaluated: 1949
HitRate@10: 0.4048
Precision@10: 0.0494
Recall@10: 0.2791
NDCG@10: 0.1667


Compare to Popularity baseline

In [58]:
POOL_LIMIT = 500
popular_k = (
    train_df.groupBy("book_id")
            .agg(functions.count("*").alias("n"), functions.avg("rating").alias("average"))
            .orderBy(functions.col("n").desc(), functions.col("average").desc())
            .limit(POOL_LIMIT)
            .select("book_id")
            .rdd.map(lambda r: r["book_id"])
            .collect()
)


In [59]:
# Recommend func for popularity baseline

def recommend_popularity(train_items, k=K_EVAL):
    seen = set(train_items)
    out = []
    for item in popular_k:
        if item not in seen:
            out.append(item)
        if len(out) >= k:
            break
    return out


In [60]:
## Evaluate populatity baseline

hits = 0
sum_precision = 0.0
sum_recall = 0.0
sum_ndcg = 0.0
n_users = 0

for row in evaluated_users_sample:
    train_items = row["train_items"]
    test_items = set(row["test_items"])

    recommendations = recommend_popularity(train_items, k=K_EVAL)
    if not recommendations:
        continue

    recommendations_set = set(recommendations)
    n_hit = len(recommendations_set.intersection(test_items))

    hits += 1 if n_hit > 0 else 0
    sum_precision += n_hit / K_EVAL
    sum_recall += n_hit / len(test_items)
    sum_ndcg += ndcg_at_k(recommendations, test_items, K_EVAL)
    n_users += 1

print("Popularity baseline:")
print("Users evaluated:", n_users)
print(f"HitRate@{K_EVAL}: {hits / n_users:.4f}")
print(f"Precision@{K_EVAL}: {sum_precision / n_users:.4f}")
print(f"Recall@{K_EVAL}: {sum_recall / n_users:.4f}")
print(f"NDCG@{K_EVAL}: {sum_ndcg / n_users:.4f}")


Popularity baseline:
Users evaluated: 2000
HitRate@10: 0.0440
Precision@10: 0.0048
Recall@10: 0.0264
NDCG@10: 0.0135


In [61]:
import re


def get_title_key(title: str, max_words: int = 5) -> str:
    if not title:
        return ""
    title = title.lower()
    title = re.sub(r"\(.*?\)", " ", title) # remove (...) content
    title = re.sub(r"\[.*?\]", " ", title) # remove [...] content
    title = title.replace("&", "and")
    title = re.sub(r"[^a-z0-9 ]+", " ", title) # drop punctuation

    TAIL_PATTERNS = [
        r"\bin prose\b",
        r"\ba novel\b",
        r"\ba memoir\b",
        r"\b(collected|complete) works\b",
        r"\b(authorized|enriched|illustrated) edition\b",
    ]

    for pat in TAIL_PATTERNS:
        title = re.sub(pat, " ", title)
    title = re.sub(r"\s+", " ", title).strip()

    # drop common “edition-ish” suffix words
    stop = {
        "unabridged","abridged","audio","audiobook","cassette","cd","edition",
        "classics","classic","collected","works","library","binding","series",
        "penguin","signet","everyman","oxford","illustrated","volume","vol",
        "part","story","being","ghost",
        "a","an","the","of","and","or","to","for","in","on","by","with","from","at"
    }
    words = [word for word in title.split() if word not in stop]

    # remove duplicate tokens while keeping order
    seen = set()
    uniq = []
    for word in words:
        if word not in seen:
            seen.add(word)
            uniq.append(word)
    words = uniq

    return " ".join(words[:max_words])




In [62]:
def normalize_articles(title_key: str) -> str:
    # drop leading articles
    for article in ("a ", "an ", "the "):
        if title_key.startswith(article):
            return title_key[len(article):]
    return title_key


In [63]:
def dedupe_recs(recommendations, book_titles, topn=10):
    seen = set()
    out = []
    for bid, pred, c in recommendations:
        title = book_titles.get(bid, "")
        key_title = normalize_articles(get_title_key(title, max_words=5))
        if not key_title or key_title in seen:
            continue
        seen.add(key_title)
        out.append((bid, pred, c, title))
        if len(out) >= topn:
            break
    return out


In [65]:
raw = recommend_for_user_centered(USER, 100)
pretty = dedupe_recs(raw, book_titles, topn=10)

for bid, pred, c, title in pretty:
    print(f"{pred:.3f} (from {c} neigh)  {bid}  -  {title}")


5.000 (from 4 neigh)  0451519582  -  Wuthering Heights (Signet classics)
5.000 (from 4 neigh)  0435126075  -  Pride & Prejudice (New Windmill)
5.000 (from 3 neigh)  B000FFQ85G  -  Jane Eyre / Wuthering Heights
5.000 (from 3 neigh)  B0007C10MS  -  The great Gatsby (Leading English literature library)
5.000 (from 3 neigh)  0140351310  -  Jane Eyre: Complete and Unabridged (Puffin Classics)
5.000 (from 3 neigh)  0451518845  -  Jane Eyre (Signet classics)
5.000 (from 3 neigh)  B000PMCF1A  -  The Catcher in the Rye [Audiobook] [Cd] [Unabridged] (Audio CD)
5.000 (from 3 neigh)  B0000632ZL  -  The Summons
5.000 (from 2 neigh)  B000NPAT6W  -  Narrative of the Life of Frederick Douglass, An American Slave. Written by Himself
5.000 (from 2 neigh)  B000NWUHR6  -  Catch-22


In [66]:
for bid, pred, c in raw[:30]:
    title = book_titles.get(bid, "")
    key_title = normalize_articles(get_title_key(title, max_words=5))
    print(f"{key_title:25s} | {title}")


wuthering heights         | Wuthering Heights (Signet classics)
pride prejudice           | Pride & Prejudice (New Windmill)
wuthering heights         | Wuthering Heights (Penguin Audiobooks)
wuthering heights         | Wuthering Heights
jane eyre wuthering heights | Jane Eyre / Wuthering Heights
pride prejudice           | Pride and Prejudice
wuthering heights         | Wuthering Heights (The Franklin Library)
great gatsby              | The great Gatsby (Leading English literature library)
jane eyre complete        | Jane Eyre: Complete and Unabridged (Puffin Classics)
jane eyre                 | Jane Eyre (Signet classics)
jane eyre                 | Jane Eyre
wuthering heights         | Wuthering Heights (New Windmill)
wuthering heights         | Wuthering Heights
wuthering heights         | Wuthering Heights
pride prejudice           | Pride and Prejudice
wuthering heights         | Wuthering Heights
wuthering heights         | Wuthering Heights.
wuthering heights         | Wuther

In [67]:
titles = [
    "A Christmas carol: In prose",
    "A Christmas Carol, in Prose: Being a Ghost Story of Christmas (Collected Works of Charles Dickens)",
    "Christmas Carol (Ladybird Classics)",
]
for tt in titles:
    print(get_title_key(tt), "|", tt)


christmas carol | A Christmas carol: In prose
christmas carol | A Christmas Carol, in Prose: Being a Ghost Story of Christmas (Collected Works of Charles Dickens)
christmas carol | Christmas Carol (Ladybird Classics)
